In [1]:
query_list_1 = ["SELECT COALESCE(a.platform, 'other') AS platform,",
 "       COALESCE(a.service, 'unknown') AS service,",
 '       COUNT(DISTINCT a.account) AS num_devs,',
 '       COUNT(DISTINCT CASE',
 '                          WHEN b.account IS NOT NULL THEN NULL',
 '                          ELSE a.account',
 '                      END) AS num_devs_xstudio_only',
 'FROM web_mobile_studio a',
 'LEFT JOIN',
 '  (SELECT account,',
 "          CONCAT_WS(',', COLLECT_SET(LOWER(service))) AS service",
 '   FROM web_mobile_studio',
 "   WHERE dt BETWEEN DATE_SUB('{run_date}', 6) AND '{run_date}'",
 '   GROUP BY account) b ON a.account = b.account',
 "AND b.service = 'studio'",
 "WHERE a.dt BETWEEN DATE_SUB('{run_date}', 6) AND '{run_date}'",
 "GROUP BY COALESCE(a.platform, 'other'),",
 "         COALESCE(a.service, 'unknown') WITH CUBE),"]

In [2]:
query_list_2 = ['SELECT s.dt,',
 "       COALESCE(em.platform, 'unknown') AS platform,",
 "       COALESCE(service_org, 'other') AS service,",
 '       account',
 'FROM sku.daily_by_account s',
 'INNER JOIN mapbox_customer_data.accounts a ON s.account = a.id',
 "AND a.dt = '{run_date}'",
 'LEFT JOIN analytics_service_endpoint_mapping em ON s.sku_id = em.sku_id',
 'AND em.in_sku IS NOT NULL',
 'AND em.parent_sku IS NULL',
 "WHERE s.dt BETWEEN DATE_SUB('{run_date}', 29) AND '{run_date}'",
 'GROUP BY 1,',
 '         2,',
 '         3,',
 '         4),']

In [15]:
for i,l in enumerate(query_list_2): print(i, l)

0 SELECT s.dt,
1        COALESCE(em.platform, 'unknown') AS platform,
2        COALESCE(service_org, 'other') AS service,
3        account
4 FROM sku.daily_by_account s
5 INNER JOIN mapbox_customer_data.accounts a ON s.account = a.id
6 AND a.dt = '{run_date}'
7 LEFT JOIN analytics_service_endpoint_mapping em ON s.sku_id = em.sku_id
8 AND em.in_sku IS NOT NULL
9 AND em.parent_sku IS NULL
10 WHERE s.dt BETWEEN DATE_SUB('{run_date}', 29) AND '{run_date}'
11 GROUP BY 1,
12          2,
13          3,
14          4),


In [46]:
query_list = query_list_2

pos_delete, pos_where = [len(query_list)-1], len(query_list)
pos_join = []
for i, line in enumerate(query_list):
    if line.startswith('ORDER') or line.startswith('GROUP'):
        pos_delete.append(i)
    elif line.startswith('FROM') and len(line.split(' ')) > 1:
        pos_join.append(i)
    elif line.startswith('FROM') and len(line.split(' ')) == 1:
        pos_join.append(i+1)
    elif line.startswith('WHERE'):
        pos_where = i
    if line.startswith('LEFT JOIN') or line.startswith('INNER JOIN') or line.startswith('FULL OUTER JOIN') or line.startswith('RIGHT JOIN'):
        for i2, line2 in enumerate(query_list[i:min(pos_where, min(pos_delete))]):
            if line2.startswith('LEFT JOIN') or line.startswith('INNER JOIN') or line.startswith('FULL OUTER JOIN') or line.startswith('RIGHT JOIN'):
                pos_join.append(i)


if min(pos_delete) == len(query_list)-1:
    pos_join.append(min(pos_delete))
else:
    pass

pos_join.extend(range(max(pos_join)+1, min(pos_where, min(pos_delete)) - 1))
    

In [47]:
min(pos_where, min(pos_delete))

10

In [48]:
pos_join

[4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 8]

In [50]:
for i,l in enumerate(query_list_1): print(i, l)

0 SELECT COALESCE(a.platform, 'other') AS platform,
1        COALESCE(a.service, 'unknown') AS service,
2        COUNT(DISTINCT a.account) AS num_devs,
3        COUNT(DISTINCT CASE
4                           WHEN b.account IS NOT NULL THEN NULL
5                           ELSE a.account
6                       END) AS num_devs_xstudio_only
7 FROM web_mobile_studio a
8 LEFT JOIN
9   (SELECT account,
10           CONCAT_WS(',', COLLECT_SET(LOWER(service))) AS service
11    FROM web_mobile_studio
12    WHERE dt BETWEEN DATE_SUB('{run_date}', 6) AND '{run_date}'
13    GROUP BY account) b ON a.account = b.account
14 AND b.service = 'studio'
15 WHERE a.dt BETWEEN DATE_SUB('{run_date}', 6) AND '{run_date}'
16 GROUP BY COALESCE(a.platform, 'other'),
17          COALESCE(a.service, 'unknown') WITH CUBE),


In [51]:
pos_join = []
for i, line in enumerate(query_list):
    if line.startswith('LEFT JOIN') or line.startswith('INNER JOIN') or line.startswith('FULL OUTER JOIN') or line.startswith('RIGHT JOIN'):
        for i2, line2 in enumerate(query_list[i:min(pos_where, min(pos_delete))]):
            if line2.startswith('LEFT JOIN') or line.startswith('INNER JOIN') or line.startswith('FULL OUTER JOIN') or line.startswith('RIGHT JOIN'):
                pos_join.append(i)
    
        
        

In [52]:
pos_join

[5, 5, 5, 5, 5, 7]